# Benchmark TME cellularity estimations on TCGAov leukocyte data

Script related to figure s3c

### Aim:
- Benchmark TME cellularity scores using leukocyte methylation data for TCGAov data

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Load packages

In [1]:
library(car)
library(MASS)
library(leaps)
library(faraway)

Loading required package: carData

Attaching package: ‘faraway’

The following objects are masked from ‘package:car’:

    logit, vif



In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/3/")

## Load TME cell estimations with Leukocyte methylation scores

### Bindea

In [3]:
bin_data <- read.csv('../../Data/3/LeukocyteScore_Bindea_TMEcells.txt',
                     sep='\t',
                     header=T,
                     row.names='X')

head(bin_data)

,LeukocyteMeth_score,activated_Dendritic_cells,B_cells,CD8_T_cells,Citotoxic_cells,Dendritic_cells,Eosinophils,immature_Dendritic_cells,Macrophages,Mast_cells,⋯,T_cells,T_helper_cells,T_central_memory_cells,T_effector_memory_cells,T_follicular_helper_cells,T_gamma_delta_cells,Th1_cells,Th17_cells,Th2_cells,Treg_cells
TCGA.04.1348.01,0.15094874,0.12080193,-0.03962222,0.2272334,0.031496941,-0.01598181,0.08082758,0.07030963,0.15797988,-0.1377551,⋯,-0.0005959854,0.2547060,0.1524806,0.1089959,0.0057093916,-0.2363975,0.0299348681,-0.04188863,0.067943526,-0.02857762
TCGA.04.1357.01,0.26794368,0.10556223,0.02903597,0.2632011,0.078787221,0.03919308,0.10824983,0.10624845,0.20255229,-0.1380420,⋯,0.1622301652,0.2568730,0.1676826,0.1400404,0.0544864070,-0.1751537,0.0423904455,0.01457536,0.006401573,-0.03299969
TCGA.04.1362.01,0.08407051,0.13333816,-0.09843530,0.2088479,-0.074348441,-0.02865189,0.11646350,0.05582148,0.13828568,-0.1291366,⋯,-0.1391578587,0.2453089,0.1745741,0.1161305,-0.0045865726,-0.2304435,0.0012755485,-0.12826950,0.024894442,-0.10347643
TCGA.04.1364.01,0.03346511,-0.05315550,-0.07195296,0.1932875,-0.163496772,-0.14344404,0.08816655,-0.01111126,0.03698071,-0.1116353,⋯,-0.2018524589,0.2623100,0.1059633,0.1152970,0.0004536352,-0.2127128,-0.0003751269,-0.13651830,0.069309195,-0.22607831
TCGA.04.1365.01,0.14147110,0.19024360,-0.09176262,0.2098552,-0.005047341,-0.03415753,0.08535212,0.07629611,0.17567990,-0.1509403,⋯,-0.0162935621,0.2732386,0.1358809,0.1195484,0.0289440950,-0.2462615,0.0330778960,-0.10256425,0.083254652,-0.08197412
TCGA.04.1514.01,0.03777766,0.07906845,-0.10770259,0.1996491,-0.174425859,-0.18025025,0.13576262,-0.00195733,0.06424089,-0.1276029,⋯,-0.1877361625,0.2657698,0.1792246,0.1068518,0.0229988609,-0.2774123,-0.0357998783,-0.08160326,0.073988042,-0.19965644


In [4]:
colnames(bin_data)

[1] "LeukocyteMeth_score"          "activated_Dendritic_cells"   
 [3] "B_cells"                      "CD8_T_cells"                 
 [5] "Citotoxic_cells"              "Dendritic_cells"             
 [7] "Eosinophils"                  "immature_Dendritic_cells"    
 [9] "Macrophages"                  "Mast_cells"                  
[11] "Neutrophils"                  "NK_CD56bright_cells"         
[13] "NK_CD56dim_cells"             "NK_cells"                    
[15] "plasmacytoid_Dendritic_cells" "T_cells"                     
[17] "T_helper_cells"               "T_central_memory_cells"      
[19] "T_effector_memory_cells"      "T_follicular_helper_cells"   
[21] "T_gamma_delta_cells"          "Th1_cells"                   
[23] "Th17_cells"                   "Th2_cells"                   
[25] "Treg_cells"

In [5]:
dim(bin_data)

[1] 305  25

### Davoli

In [6]:
dav_data <- read.csv('../../Data/3/LeukocyteScore_Davoli_TMEcells.txt',
                     sep='\t',
                     header=T,
                     row.names='X')

head(dav_data)

,LeukocyteMeth_score,CD4_mature,CD8_effector,NK_cells,B_cells,T_regs,Dendritics,CD8_effector_NK_cells,Macrophages,Macrophages_M2,Macrophages_M1
TCGA.04.1348.01,0.15094874,0.02265669,-0.13368553,-0.139780840,-0.045055042,-0.01413145,0.13264289,-0.01007489,0.413100804,0.26219802,0.3699730
TCGA.04.1357.01,0.26794368,0.10361223,0.05135761,0.004286589,0.148263482,-0.01467961,0.14826799,0.12913414,0.450267807,0.30454248,0.4334608
TCGA.04.1362.01,0.08407051,-0.12438993,-0.36916463,-0.346158705,-0.209139765,-0.09584074,-0.04967173,-0.21720120,0.333927607,0.16942322,0.1708495
TCGA.04.1364.01,0.03346511,-0.18234025,-0.43643981,-0.467785794,-0.127646643,-0.22223507,-0.09767746,-0.35831776,-0.022455228,-0.08036410,-0.1122478
TCGA.04.1365.01,0.14147110,-0.04229328,-0.15522688,-0.146162797,-0.005805204,-0.10821798,0.12214756,-0.04539836,0.390748172,0.21386171,0.3483508
TCGA.04.1514.01,0.03777766,-0.14808391,-0.35410255,-0.435578023,-0.237304750,-0.10345609,-0.04896112,-0.31855763,0.001551493,-0.09999489,-0.1151634


In [7]:
colnames(dav_data)

[1] "LeukocyteMeth_score"   "CD4_mature"            "CD8_effector"         
 [4] "NK_cells"              "B_cells"               "T_regs"               
 [7] "Dendritics"            "CD8_effector_NK_cells" "Macrophages"          
[10] "Macrophages_M2"        "Macrophages_M1"

In [8]:
dim(dav_data)

[1] 305  11

### Danaher

In [9]:
dan_data <- read.csv('../../Data/3/LeukocyteScore_Danaher_TMEcells.txt',
                     sep='\t',
                     header=T,
                     row.names='X')

head(dan_data)

,LeukocyteMeth_score,B.cells,CD45,CD8.T.cells,Cytotoxic.cells,DC,Exhausted.CD8,Macrophages,Mast.cells,Neutrophils,NK.CD56dim.cells,NK.cells,T.cells,Th1.cells,Treg
TCGA.04.1348.01,0.15094874,-0.21377187,0.237833263,0.03857399,-0.004140244,-0.1367343,-0.06245932,0.2105348,-0.3071445,-0.1885067,-0.2363422,-0.2043080,-0.04364003,-0.1831358,-0.02734871
TCGA.04.1357.01,0.26794368,-0.08940436,0.422714769,0.23643827,0.072817038,-0.1475562,0.03239466,0.2311399,-0.3179682,-0.1602179,-0.2190440,-0.1668824,0.09784111,-0.1087601,-0.03158062
TCGA.04.1362.01,0.08407051,-0.34364185,0.006612357,-0.22610982,-0.209049699,-0.2321549,-0.20919985,0.0509568,-0.3356827,-0.2121856,-0.3636091,-0.3125212,-0.23293336,-0.3198265,-0.09902666
TCGA.04.1364.01,0.03346511,-0.18934390,-0.248201439,-0.23943913,-0.334968312,-0.2631513,-0.31065397,-0.1877227,-0.3866473,-0.3229375,-0.4504338,-0.2919417,-0.31326032,-0.5289886,-0.21635633
TCGA.04.1365.01,0.14147110,-0.23129310,0.175730004,-0.02170060,-0.037648485,-0.1806338,-0.03781317,0.1635445,-0.3390506,-0.1458923,-0.2582365,-0.2590599,-0.04973714,-0.1999048,-0.07844901
TCGA.04.1514.01,0.03777766,-0.28738393,-0.206887431,-0.22293833,-0.338726748,-0.2249112,-0.25213861,-0.1572274,-0.3872197,-0.3122133,-0.4357587,-0.4259567,-0.31991352,-0.3520419,-0.19107067


In [10]:
colnames(dan_data)

[1] "LeukocyteMeth_score" "B.cells"             "CD45"               
 [4] "CD8.T.cells"         "Cytotoxic.cells"     "DC"                 
 [7] "Exhausted.CD8"       "Macrophages"         "Mast.cells"         
[10] "Neutrophils"         "NK.CD56dim.cells"    "NK.cells"           
[13] "T.cells"             "Th1.cells"           "Treg"

In [11]:
dim(dan_data)

[1] 305  15

### ConsensusTME

In [12]:
con_data <- read.csv('../../Data/3/LeukocyteScore_ConsensusTME_TMEcells.txt',
                     sep='\t',
                     header=T,
                     row.names='X')

head(con_data)

,LeukocyteMeth_score,B_cells,Cytotoxic_cells,Dendritic_cells,Endothelial,Eosinophils,Fibroblasts,Macrophages,Macrophages_M1,Macrophages_M2,Mast_cells,Monocytes,NK_cells,Neutrophils,Plasma_cells,T_cells_CD4,T_cells_CD8,T_cells_gamma_delta,T_regulatory_cells
TCGA.04.1348.01,0.15094874,0.11609382,0.01519822,0.17787153,-0.02197883,0.16327315,-0.02636739,0.30835996,0.26594722,0.26372822,0.22779935,0.15637171,0.04008748,-0.04270634,0.3281884,0.08150827,0.09753041,0.06461224,0.1113211
TCGA.04.1357.01,0.26794368,0.22290548,0.14255561,0.26478408,0.01835045,0.25433465,0.12083934,0.33543100,0.32518087,0.35331336,0.31541603,0.23793736,0.20089126,0.02280329,0.3925301,0.23510870,0.24758509,0.21513726,0.2640703
TCGA.04.1362.01,0.08407051,-0.05337479,-0.31383134,-0.01250251,-0.10314986,-0.01236931,-0.08078278,0.14003631,0.07572393,0.07619906,-0.02492258,0.03764975,-0.19824721,-0.15723850,0.2362371,-0.12795204,-0.12669652,-0.17325147,-0.1712345
TCGA.04.1364.01,0.03346511,-0.24205622,-0.47925724,-0.24144746,-0.13300333,-0.17731889,-0.10190933,-0.04109388,-0.12067643,-0.09100734,-0.12599570,-0.19670629,-0.39921650,-0.36128105,0.1175052,-0.29898201,-0.28334875,-0.28693058,-0.3985474
TCGA.04.1365.01,0.14147110,0.04399216,-0.05918070,0.12695363,-0.03542567,0.13760869,-0.02378641,0.26573980,0.24497140,0.21709724,0.19281831,0.12738330,-0.01475235,-0.02982533,0.3333733,0.02708843,0.03979512,0.03105127,0.0398150
TCGA.04.1514.01,0.03777766,-0.22959159,-0.50135244,-0.21268284,-0.03784834,-0.14256235,0.04346422,-0.05456973,-0.11654632,-0.16530375,-0.25167799,-0.19286174,-0.40757855,-0.33511077,0.1331253,-0.28737986,-0.27382453,-0.31684395,-0.3782873


In [13]:
colnames(con_data)

[1] "LeukocyteMeth_score" "B_cells"             "Cytotoxic_cells"    
 [4] "Dendritic_cells"     "Endothelial"         "Eosinophils"        
 [7] "Fibroblasts"         "Macrophages"         "Macrophages_M1"     
[10] "Macrophages_M2"      "Mast_cells"          "Monocytes"          
[13] "NK_cells"            "Neutrophils"         "Plasma_cells"       
[16] "T_cells_CD4"         "T_cells_CD8"         "T_cells_gamma_delta"
[19] "T_regulatory_cells"

#### Remove non-leukocyte cells
<p> 
Our response variable is leukocyte methylation data, so any non-relevant predictor variables (i.e. cells other than leukocytes) are excluded from the analysis.
</p>

In [14]:
drops <- c('endothelial','fibroblasts')
con_data <- con_data[ , !(names(con_data) %in% drops)]

colnames(con_data)

[1] "LeukocyteMeth_score" "B_cells"             "Cytotoxic_cells"    
 [4] "Dendritic_cells"     "Endothelial"         "Eosinophils"        
 [7] "Fibroblasts"         "Macrophages"         "Macrophages_M1"     
[10] "Macrophages_M2"      "Mast_cells"          "Monocytes"          
[13] "NK_cells"            "Neutrophils"         "Plasma_cells"       
[16] "T_cells_CD4"         "T_cells_CD8"         "T_cells_gamma_delta"
[19] "T_regulatory_cells"

In [15]:
dim(con_data)

[1] 305  19

### CIBERSORT

In [16]:
cib_data <- read.csv('../../Data/3/LeukocyteScore_cibersort_TMEcells.txt',
                     sep='\t',
                     header=T,
                     row.names='X')

head(cib_data)

,LeukocyteMeth_score,B.cells.naive,B.cells.memory,Plasma.cells,T.cells.CD8,T.cells.CD4.naive,T.cells.CD4.memory.resting,T.cells.CD4.memory.activated,T.cells.follicular.helper,T.cells.regulatory..Tregs.,⋯,Monocytes,Macrophages.M0,Macrophages.M1,Macrophages.M2,Dendritic.cells.resting,Dendritic.cells.activated,Mast.cells.resting,Mast.cells.activated,Eosinophils,Neutrophils
TCGA.04.1348.01,0.15094874,0.000795189,0.0008304125,0.000000e+00,0.02586837,0,0.038002807,0,0.043519385,0.026831901,⋯,0.005155625,0.10498343,0.02423711,0.062197526,0.000000000,0.00000000,0.000000e+00,0.014233638,0.000000000,0.000000000
TCGA.04.1357.01,0.26794368,0.015474451,0.0000000000,0.000000e+00,0.08310068,0,0.120940104,0,0.053591895,0.052899699,⋯,0.006448388,0.05762259,0.06234337,0.124706843,0.001035095,0.00000000,1.460025e-02,0.000000000,0.000000000,0.000000000
TCGA.04.1362.01,0.08407051,0.008684441,0.0000000000,5.751311e-05,0.00000000,0,0.049975061,0,0.010517768,0.000000000,⋯,0.016485859,0.00000000,0.00450224,0.058744928,0.000000000,0.01751784,0.000000e+00,0.001998067,0.000000000,0.000000000
TCGA.04.1364.01,0.03346511,0.018039723,0.0000000000,1.023013e-03,0.00000000,0,0.004297134,0,0.001651301,0.002840003,⋯,0.000000000,0.01707628,0.00000000,0.005596468,0.000000000,0.00000000,4.552176e-05,0.000000000,0.000000000,0.000000000
TCGA.04.1365.01,0.14147110,0.001747266,0.0002290281,0.000000e+00,0.00000000,0,0.072325307,0,0.048944230,0.020026142,⋯,0.003811225,0.08054884,0.05469904,0.034942770,0.000000000,0.00000000,2.676272e-03,0.000000000,0.000000000,0.000605261
TCGA.04.1514.01,0.03777766,0.000000000,0.0000000000,0.000000e+00,0.00000000,0,0.018580530,0,0.016439241,0.005836911,⋯,0.007142607,0.01378685,0.00000000,0.000000000,0.000000000,0.00000000,0.000000e+00,0.000000000,0.003241758,0.000000000


In [17]:
colnames(cib_data)

[1] "LeukocyteMeth_score"          "B.cells.naive"               
 [3] "B.cells.memory"               "Plasma.cells"                
 [5] "T.cells.CD8"                  "T.cells.CD4.naive"           
 [7] "T.cells.CD4.memory.resting"   "T.cells.CD4.memory.activated"
 [9] "T.cells.follicular.helper"    "T.cells.regulatory..Tregs."  
[11] "T.cells.gamma.delta"          "NK.cells.resting"            
[13] "NK.cells.activated"           "Monocytes"                   
[15] "Macrophages.M0"               "Macrophages.M1"              
[17] "Macrophages.M2"               "Dendritic.cells.resting"     
[19] "Dendritic.cells.activated"    "Mast.cells.resting"          
[21] "Mast.cells.activated"         "Eosinophils"                 
[23] "Neutrophils"

In [18]:
dim(cib_data)

[1] 305  23

### MCPcounter

In [19]:
mcp_data <- read.csv('../../Data/3/LeukocyteScore_mcp_TMEcells.txt',
                     sep='\t',
                     header=T,
                     row.names='X')

head(mcp_data)

,LeukocyteMeth_score,T.cells,CD8.T.cells,Cytotoxic.lymphocytes,NK.cells,B.lineage,Monocytic.lineage,Myeloid.dendritic.cells,Neutrophils,Endothelial.cells,Fibroblasts
TCGA.04.1348.01,0.15094874,140.37951,82.0113,122.44650,5.310562,61.40764,552.41961,34.31030,185.0318,227.8419,5541.237
TCGA.04.1357.01,0.26794368,284.69122,326.8229,289.74794,10.823550,220.78453,897.16706,154.29686,245.2691,204.5117,9558.854
TCGA.04.1362.01,0.08407051,217.39259,5.2953,20.04664,5.088475,38.76356,313.79614,12.97356,193.2578,257.9961,3400.404
TCGA.04.1364.01,0.03346511,41.82687,19.9291,1.87780,4.187238,98.47945,114.84979,5.85152,158.8109,236.2052,2706.701
TCGA.04.1365.01,0.14147110,167.17966,66.2950,107.27739,7.533512,92.02199,462.09216,40.43996,185.9475,201.2723,3607.766
TCGA.04.1514.01,0.03777766,80.24919,4.3178,11.22624,1.619175,155.47641,87.30159,11.28382,281.8845,363.7016,13060.485


In [20]:
colnames(mcp_data)

[1] "LeukocyteMeth_score"     "T.cells"                
 [3] "CD8.T.cells"             "Cytotoxic.lymphocytes"  
 [5] "NK.cells"                "B.lineage"              
 [7] "Monocytic.lineage"       "Myeloid.dendritic.cells"
 [9] "Neutrophils"             "Endothelial.cells"      
[11] "Fibroblasts"

#### Remove non-leukocyte cells

In [21]:
drops <- c('Endothelial.cells','Fibroblasts')
mcp_data <- mcp_data[ , !(names(mcp_data) %in% drops)]

colnames(mcp_data)

[1] "LeukocyteMeth_score"     "T.cells"                
[3] "CD8.T.cells"             "Cytotoxic.lymphocytes"  
[5] "NK.cells"                "B.lineage"              
[7] "Monocytic.lineage"       "Myeloid.dendritic.cells"
[9] "Neutrophils"

In [22]:
dim(mcp_data)

[1] 305   9

### TIMER

In [23]:
tim_data <- read.csv('../../Data/3/LeukocyteScore_timer_TMEcells.txt',
                     sep='\t',
                     header=T,
                     row.names='sampleID')

head(tim_data)

,LeukocyteMeth_score,B_cell,T_cell.CD4,T_cell.CD8,Neutrophil,Macrophage,DC
TCGA.04.1348.01,0.15094874,0.28486857,0.20634290,0.13801796,0.18701448,0.00000000,0.5759300
TCGA.04.1357.01,0.26794368,0.60525898,0.25946178,0.02422188,0.10000450,0.00000000,0.8067421
TCGA.04.1362.01,0.08407051,0.15997653,0.14442067,0.26335778,0.10042916,0.10724381,0.3387293
TCGA.04.1364.01,0.03346511,0.14925784,0.21527554,0.02234241,0.00000000,0.04167787,0.1693009
TCGA.04.1365.01,0.14147110,0.04125805,0.08668255,0.28274628,0.19528997,0.00000000,0.5331854
TCGA.04.1514.01,0.03777766,0.00000000,0.08187040,0.11963303,0.03853453,0.16070039,0.1446298


In [24]:
colnames(tim_data)

[1] "LeukocyteMeth_score" "B_cell"              "T_cell.CD4"         
[4] "T_cell.CD8"          "Neutrophil"          "Macrophage"         
[7] "DC"

In [25]:
dim(tim_data)

[1] 305   7

### xCELL

In [26]:
xce_data <- read.csv('../../Data/3/LeukocyteScore_xcell_TMEcells.txt',
                     sep='\t',
                     header=T,
                     row.names='X')

head(xce_data)

,LeukocyteMeth_score,Adipocytes,Astrocytes,B.cells,Basophils,CD4..T.cells,CD4..Tcm,CD4..Tem,CD4..memory.T.cells,CD4..naive.T.cells,⋯,Th2.cells,Tregs,aDC,cDC,iDC,ly.Endothelial.cells,mv.Endothelial.cells,naive.B.cells,pDC,pro.B.cells
TCGA.04.1348.01,0.15094874,0.0000,0.0494,0.0535,0.1007,0,0.0000,0.0000,0.0074,0.0000,⋯,0.1211,0.0000,0.3265,0.0287,0.0000,0.0000,0.0098,0,0.0199,0.0442
TCGA.04.1357.01,0.26794368,0.0000,0.0301,0.1330,0.0486,0,0.0034,0.0458,0.0087,0.0227,⋯,0.0081,0.0057,0.3852,0.1140,0.0000,0.0000,0.0037,0,0.0382,0.0015
TCGA.04.1362.01,0.08407051,0.0000,0.0506,0.0039,0.0393,0,0.0239,0.0000,0.0000,0.0000,⋯,0.0780,0.0000,0.1563,0.0125,0.0000,0.0019,0.0032,0,0.0070,0.0116
TCGA.04.1364.01,0.03346511,0.0016,0.0654,0.0000,0.0452,0,0.0038,0.0000,0.0000,0.0000,⋯,0.1076,0.0000,0.0000,0.0000,0.0000,0.0021,0.0117,0,0.0000,0.0390
TCGA.04.1365.01,0.14147110,0.0000,0.0612,0.0179,0.0906,0,0.0000,0.0000,0.0121,0.0000,⋯,0.1709,0.0000,0.2709,0.0289,0.0000,0.0000,0.0039,0,0.0155,0.0276
TCGA.04.1514.01,0.03777766,0.0000,0.1008,0.0088,0.0453,0,0.0245,0.0000,0.0000,0.0000,⋯,0.0388,0.0000,0.0000,0.0043,0.0571,0.0007,0.0000,0,0.0000,0.0212


In [27]:
colnames(xce_data)

[1] "LeukocyteMeth_score"           "Adipocytes"                   
 [3] "Astrocytes"                    "B.cells"                      
 [5] "Basophils"                     "CD4..T.cells"                 
 [7] "CD4..Tcm"                      "CD4..Tem"                     
 [9] "CD4..memory.T.cells"           "CD4..naive.T.cells"           
[11] "CD8..T.cells"                  "CD8..Tcm"                     
[13] "CD8..Tem"                      "CD8..naive.T.cells"           
[15] "CLP"                           "CMP"                          
[17] "Chondrocytes"                  "Class.switched.memory.B.cells"
[19] "DC"                            "Endothelial.cells"            
[21] "Eosinophils"                   "Epithelial.cells"             
[23] "Erythrocytes"                  "Fibroblasts"                  
[25] "GMP"                           "HSC"                          
[27] "Hepatocytes"                   "Keratinocytes"                
[29] "MEP"                           "MPP"                          
[31] "MSC"                           "Macrophages"                  
[33] "Macrophages.M1"                "Macrophages.M2"               
[35] "Mast.cells"                    "Megakaryocytes"               
[37] "Melanocytes"                   "Memory.B.cells"               
[39] "Mesangial.cells"               "Monocytes"                    
[41] "Myocytes"                      "NK.cells"                     
[43] "NKT"                           "Neurons"                      
[45] "Neutrophils"                   "Osteoblast"                   
[47] "Pericytes"                     "Plasma.cells"                 
[49] "Platelets"                     "Preadipocytes"                
[51] "Sebocytes"                     "Skeletal.muscle"              
[53] "Smooth.muscle"                 "Tgd.cells"                    
[55] "Th1.cells"                     "Th2.cells"                    
[57] "Tregs"                         "aDC"                          
[59] "cDC"                           "iDC"                          
[61] "ly.Endothelial.cells"          "mv.Endothelial.cells"         
[63] "naive.B.cells"                 "pDC"                          
[65] "pro.B.cells"

#### Remove non-leukocyte cells

In [28]:
drops <- c('Adipocytes','Astrocytes','Chondrocytes','CLP','CMP','Endothelial.cells','Epithelial.cells','Erythrocytes',
           'Fibroblasts','Hepatocytes','HSC','Keratinocytes','ly.Endothelial.cells','Megakaryocytes','Melanocytes','MEP',
           'Mesangial.cells','MPP','MSC','mv.Endothelial.cells','Myocytes','Neurons','Osteoblast','Pericytes','Platelets',
           'Preadipocytes','Sebocytes','Skeletal.muscle','Smooth.muscle','ImmuneScore','StromaScore','MicroenvironmentScore')

xce_data <- xce_data[ , !(names(xce_data) %in% drops)]

colnames(xce_data)


[1] "LeukocyteMeth_score"           "B.cells"                      
 [3] "Basophils"                     "CD4..T.cells"                 
 [5] "CD4..Tcm"                      "CD4..Tem"                     
 [7] "CD4..memory.T.cells"           "CD4..naive.T.cells"           
 [9] "CD8..T.cells"                  "CD8..Tcm"                     
[11] "CD8..Tem"                      "CD8..naive.T.cells"           
[13] "Class.switched.memory.B.cells" "DC"                           
[15] "Eosinophils"                   "GMP"                          
[17] "Macrophages"                   "Macrophages.M1"               
[19] "Macrophages.M2"                "Mast.cells"                   
[21] "Memory.B.cells"                "Monocytes"                    
[23] "NK.cells"                      "NKT"                          
[25] "Neutrophils"                   "Plasma.cells"                 
[27] "Tgd.cells"                     "Th1.cells"                    
[29] "Th2.cells"                     "Tregs"                        
[31] "aDC"                           "cDC"                          
[33] "iDC"                           "naive.B.cells"                
[35] "pDC"                           "pro.B.cells"

In [29]:
dim(xce_data)

[1] 305  36

## Do multiple regression analysis

In [30]:
bin_m1<-lm(LeukocyteMeth_score~., data=bin_data)
dav_m1<-lm(LeukocyteMeth_score~., data=dav_data)
dan_m1<-lm(LeukocyteMeth_score~., data=dan_data)
con_m1<-lm(LeukocyteMeth_score~., data=con_data)
cib_m1<-lm(LeukocyteMeth_score~., data=cib_data)
mcp_m1<-lm(LeukocyteMeth_score~., data=mcp_data)
tim_m1<-lm(LeukocyteMeth_score~., data=tim_data)
xce_m1<-lm(LeukocyteMeth_score~., data=xce_data)

### Evaluate normality of residuals

In [31]:
p_values = c(
    shapiro.test(bin_m1$res)$p,
    shapiro.test(dav_m1$res)$p,
    shapiro.test(dan_m1$res)$p,
    shapiro.test(con_m1$res)$p,
    shapiro.test(cib_m1$res)$p,
    shapiro.test(mcp_m1$res)$p,
    shapiro.test(tim_m1$res)$p,
    shapiro.test(xce_m1$res)$p
)

adj.p_values <- c(p.adjust(p_values, method='fdr', n=length(p_values)))

In [32]:
methods <- c(
    'Bindea',
    'Davoli',
    'Danaher',
    'ConsensusTME',
    'Cibersort',
    'MCP',
    'TIMER',
    'xCell'
)

adjpvals <- data.frame(
    'Method'=methods,
    'Norm_Res_Adj_pvals'=adj.p_values
)

adjpvals

Method,Norm_Res_Adj_pvals
Bindea,1.536639e-09
Davoli,6.610000e-09
Danaher,3.596906e-08
ConsensusTME,6.610000e-09
Cibersort,5.103905e-08
MCP,3.082377e-11
TIMER,1.939403e-04
xCell,4.462045e-08


### Note: Normality of residuals assumption

<p>
    Since n is large (n=305), due to the central limit theorem, normality assumptions are not strictly required.</p>
<p>    
    See: https://www.ncbi.nlm.nih.gov/pubmed/11910059 </p>
<p>
    However, we can test both scenarios:</p>
<p>    
    1) Residuals' normality assumption is completely ignored</p>
<p>        
    2) Residuals' normality assumption is approached</p>

## 1) Normality of residuals is ignored

In [33]:
methods <- c(
    'Bindea',
    'Davoli',
    'Danaher',
    'ConsensusTME',
    'Cibersort',
    'MCP',
    'TIMER',
    'xCell'
)

adjrsqr <- c(
    summary(bin_m1)$adj.r.squared,
    summary(dav_m1)$adj.r.squared,
    summary(dan_m1)$adj.r.squared,
    summary(con_m1)$adj.r.squared,
    summary(cib_m1)$adj.r.squared,
    summary(mcp_m1)$adj.r.squared,
    summary(tim_m1)$adj.r.squared,
    summary(xce_m1)$adj.r.squared   
)

aic <- AIC(
    bin_m1,
    dav_m1,
    dan_m1,
    con_m1,
    cib_m1,
    mcp_m1,
    tim_m1,
    xce_m1
)$AIC

bic <- BIC(
    bin_m1,
    dav_m1,
    dan_m1,
    con_m1,
    cib_m1,
    mcp_m1,
    tim_m1,
    xce_m1
)$BIC

In [34]:
adjrsqr <- data.frame(
    'Method'=methods,
    'Norm_Res_Adj_pvals'=adj.p_values,
    'Adj.r.sqr'=adjrsqr,
    'AIC'=aic,
    'BIC'=bic
)

adjrsqr

Method,Norm_Res_Adj_pvals,Adj.r.sqr,AIC,BIC
Bindea,1.536639e-09,0.6272973,-994.4575,-897.7294
Davoli,6.610000e-09,0.6118627,-995.2002,-950.5565
Danaher,3.596906e-08,0.6316077,-1007.3026,-947.7776
ConsensusTME,6.610000e-09,0.6656041,-1033.0697,-958.6635
Cibersort,5.103905e-08,0.6948601,-1057.2899,-968.0024
MCP,3.082377e-11,0.5515239,-953.0610,-915.8579
TIMER,1.939403e-04,0.5909792,-983.0944,-953.3319
xCell,4.462045e-08,0.6699459,-1021.7463,-884.0948


### Save dataframe

In [35]:
write.table(adjrsqr,
            file='../../Data/3/TCGAov_methods_benchamrk_res_notNormal.txt',
            sep='\t',
            row.names=F,
            col.names=T)

## 2) Normality of residuals is approached

### Transfrom response variable using boxcox method

In [ ]:
# conseusns boxcox
con_m1_bxcx<-boxcox(con_m1,plotit=F)
lambda_con <- con_m1_bxcx$x[con_m1_bxcx$y==max(con_m1_bxcx$y)]

# bindea boxcox
bin_m1_bxcx<-boxcox(bin_m1,plotit=F)
lambda_bin <- bin_m1_bxcx$x[bin_m1_bxcx$y==max(bin_m1_bxcx$y)]

# davoli boxcox
dav_m1_bxcx<-boxcox(dav_m1,plotit=F)
lambda_dav <- dav_m1_bxcx$x[dav_m1_bxcx$y==max(dav_m1_bxcx$y)]

# danaher boxcox
dan_m1_bxcx<-boxcox(dan_m1,plotit=F)
lambda_dan <- dan_m1_bxcx$x[dan_m1_bxcx$y==max(dan_m1_bxcx$y)]

# mcp boxcox
mcp_m1_bxcx<-boxcox(mcp_m1,plotit=F)
lambda_mcp <- mcp_m1_bxcx$x[mcp_m1_bxcx$y==max(mcp_m1_bxcx$y)]

# cibersort boxcox
cib_m1_bxcx<-boxcox(cib_m1,plotit=F)
lambda_cib <- cib_m1_bxcx$x[cib_m1_bxcx$y==max(cib_m1_bxcx$y)]

# timer boxcox
tim_m1_bxcx<-boxcox(tim_m1,plotit=F)
lambda_tim <- tim_m1_bxcx$x[tim_m1_bxcx$y==max(tim_m1_bxcx$y)]

# xcell boxcox
xce_m1_bxcx<-boxcox(xce_m1,plotit=F)
lambda_xce <- xce_m1_bxcx$x[xce_m1_bxcx$y==max(xce_m1_bxcx$y)]

In [ ]:
lambda<-median(c(
    lambda_con,
    lambda_bin,
    lambda_dav,
    lambda_dan,
    lambda_mcp,
    lambda_cib,
    lambda_tim,
    lambda_xce))

lambda

### Save untransformed leukocyte methylation score

In [36]:
con_y<-con_data$LeukocyteMeth_score
bin_y<-bin_data$LeukocyteMeth_score
dan_y<-dan_data$LeukocyteMeth_score
dav_y<-dav_data$LeukocyteMeth_score
mcp_y<-mcp_data$LeukocyteMeth_score
cib_y<-cib_data$LeukocyteMeth_score
tim_y<-tim_data$LeukocyteMeth_score
xce_y<-xce_data$LeukocyteMeth_score

### Transform and assign response variable
#### For lamda != 0 use: (y^lambda -1) / lambda
#### For lambda == 0 use: log(y)

In [ ]:
con_data$LeukocyteMeth_score<-(con_y^lambda - 1) / lambda
bin_data$LeukocyteMeth_score<-(bin_y^lambda - 1) / lambda
dav_data$LeukocyteMeth_score<-(dav_y^lambda - 1) / lambda
dan_data$LeukocyteMeth_score<-(dan_y^lambda - 1) / lambda
mcp_data$LeukocyteMeth_score<-(mcp_y^lambda - 1) / lambda
cib_data$LeukocyteMeth_score<-(cib_y^lambda - 1) / lambda
tim_data$LeukocyteMeth_score<-(tim_y^lambda - 1) / lambda
xce_data$LeukocyteMeth_score<-(xce_y^lambda - 1) / lambda

In [37]:
con_data$LeukocyteMeth_score<-(con_y^(1/3))
bin_data$LeukocyteMeth_score<-(bin_y^(1/3))
dav_data$LeukocyteMeth_score<-(dav_y^(1/3))
dan_data$LeukocyteMeth_score<-(dan_y^(1/3))
mcp_data$LeukocyteMeth_score<-(mcp_y^(1/3))
cib_data$LeukocyteMeth_score<-(cib_y^(1/3))
tim_data$LeukocyteMeth_score<-(tim_y^(1/3))
xce_data$LeukocyteMeth_score<-(xce_y^(1/3))

## Perform multiple linear regression with transformed response variable

In [38]:
bin_m1_lambda<-lm(LeukocyteMeth_score~., data=bin_data)
dav_m1_lambda<-lm(LeukocyteMeth_score~., data=dav_data)
dan_m1_lambda<-lm(LeukocyteMeth_score~., data=dan_data)
con_m1_lambda<-lm(LeukocyteMeth_score~., data=con_data)
cib_m1_lambda<-lm(LeukocyteMeth_score~., data=cib_data)
mcp_m1_lambda<-lm(LeukocyteMeth_score~., data=mcp_data)
tim_m1_lambda<-lm(LeukocyteMeth_score~., data=tim_data)
xce_m1_lambda<-lm(LeukocyteMeth_score~., data=xce_data)

### Evaluate normality of residuals

In [39]:
p_values_lambda = c(
    shapiro.test(bin_m1_lambda$res)$p,
    shapiro.test(dav_m1_lambda$res)$p,
    shapiro.test(dan_m1_lambda$res)$p,
    shapiro.test(con_m1_lambda$res)$p,
    shapiro.test(cib_m1_lambda$res)$p,
    shapiro.test(mcp_m1_lambda$res)$p,
    shapiro.test(tim_m1_lambda$res)$p,
    shapiro.test(xce_m1_lambda$res)$p
)

adj.p_values_lambda <- c(p.adjust(
    p_values_lambda,
    method='fdr',
    n=length(p_values_lambda)))

In [40]:
methods <- c(
    'Bindea',
    'Davoli',
    'Danaher',
    'ConsensusTME',
    'Cibersort',
    'MCP',
    'TIMER',
    'xCell'
)

adjpvals_lambda <- data.frame(
    'Method'=methods,
    'Norm_Res_Adj_pvals_lambda'=adj.p_values_lambda
)

adjpvals_lambda

Method,Norm_Res_Adj_pvals_lambda
Bindea,0.008881381
Davoli,0.006043508
Danaher,0.016790879
ConsensusTME,0.116255555
Cibersort,0.068940401
MCP,0.001291623
TIMER,0.876255432
xCell,0.112684625


In [41]:
methods <- c(
    'Bindea',
    'Davoli',
    'Danaher',
    'ConsensusTME',
    'Cibersort',
    'MCP',
    'TIMER',
    'xCell'
)

adjrsqr_lambda <- c(
    summary(bin_m1_lambda)$adj.r.squared,
    summary(dav_m1_lambda)$adj.r.squared,
    summary(dan_m1_lambda)$adj.r.squared,
    summary(con_m1_lambda)$adj.r.squared,
    summary(cib_m1_lambda)$adj.r.squared,
    summary(mcp_m1_lambda)$adj.r.squared,
    summary(tim_m1_lambda)$adj.r.squared,
    summary(xce_m1_lambda)$adj.r.squared   
)

aic_lambda <- AIC(
    bin_m1_lambda,
    dav_m1_lambda,
    dan_m1_lambda,
    con_m1_lambda,
    cib_m1_lambda,
    mcp_m1_lambda,
    tim_m1_lambda,
    xce_m1_lambda
)$AIC

bic_lambda <- BIC(
    bin_m1_lambda,
    dav_m1_lambda,
    dan_m1_lambda,
    con_m1_lambda,
    cib_m1_lambda,
    mcp_m1_lambda,
    tim_m1_lambda,
    xce_m1_lambda
)$BIC

In [42]:
results_lambda <- data.frame(
    'Method'   = methods,
    'Norm_Res_Adj_pvals_lambda'= adj.p_values_lambda,
    'Adj.r.sqr'= adjrsqr_lambda,
    'AIC'      = aic_lambda,
    'BIC'      = bic_lambda
)

results_lambda

Method,Norm_Res_Adj_pvals_lambda,Adj.r.sqr,AIC,BIC
Bindea,0.008881381,0.6831239,-855.3969,-758.6688
Davoli,0.006043508,0.6700824,-856.2147,-811.5709
Danaher,0.016790879,0.6733055,-855.3871,-795.8621
ConsensusTME,0.116255555,0.7095478,-887.4872,-813.0810
Cibersort,0.068940401,0.6817078,-855.8661,-766.5786
MCP,0.001291623,0.5341770,-752.9332,-715.7300
TIMER,0.876255432,0.5983720,-800.1046,-770.3421
xCell,0.112684625,0.6622572,-826.1698,-688.5182


In [ ]:
summary(con_m1)

In [ ]:
library( broom )

In [ ]:
write.table( tidy( con_m1 ) , "../../Data/3/ConsensusTME_notNormal.txt" , sep='\t')
#write.csv( glance( con_m1_lambda ) , "an.csv" )

### Save dataframe

In [ ]:
write.table(results_lambda,
            file='../../Data/3/TCGAov_methods_benchamrk_res_nearNormal.txt',
            sep='\t',
            row.names=F,
            col.names=T)

# End script